In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, Normalizer, MinMaxScaler

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

import plotly.express as px

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.feature_selection import RFECV

from datetime import datetime

pd.set_option('display.max_rows', 5000)

<H1>External feature integration

In [2]:
df=pd.read_csv('../data/zri_acs_bikeshare_merged.csv')

In [3]:
df['year-month']=df['datetime'].apply(lambda x: x[0:7])

<H1>Standardization

In [4]:
#df=df.drop('zip', axis=1)

for zipcode in df['zip'].unique():
    globals()[f"minmax_{zipcode}"]=MinMaxScaler(copy=False)
    df_filtered=df[df['zip']==zipcode]
    df.loc[df['zip']==zipcode,'zri']=globals()[f"minmax_{zipcode}"].fit_transform(df_filtered[['zri']])

<H1>Lag development

In [5]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [6]:
def laggenerator_diff(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+1}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+1)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+1}']=0
    return df

In [7]:
def laggenerator_diff12(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+11]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+11}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+11)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+11}']=0
    return df

In [8]:
def laggenerator_diff6(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+6}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+6)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+6}']=0
    return df

In [9]:
def laggenerator_MA(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+4]
    
    df.loc[:,f'{colname}_MA{i}']=df.loc[:,f'{colname}'].shift(i)-(df.loc[:,f'{colname}'].shift(i)+df.loc[:,f'{colname}'].shift(i+1)+df.loc[:,f'{colname}'].shift(i+2)+df.loc[:,f'{colname}'].shift(i+3)+df.loc[:,f'{colname}'].shift(i+4))/5
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_MA{i}']=0
    return df

In [10]:
def laggenerator_mean(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_mean{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_mean{i}']+=df.loc[:,f'{colname}'].shift(x)
    df.loc[:,f'{colname}_mean{i}']=df.loc[:,f'{colname}_mean{i}']/i
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_mean{i}']=0
    return df

In [11]:
def laggenerator_ewm(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_ewm{i}']=0
    weightsum=0
    for y in range(1,i):
        weight=i-y
        weightsum+=weight
    for x in range(1,i):
        weight=i-x
        df.loc[:,f'{colname}_ewm{i}']+=df.loc[:,f'{colname}'].shift(x)*weight/weightsum
    df.loc[:,f'{colname}_ewm{i}']=df.loc[:,f'{colname}_ewm{i}']
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_ewm{i}']=0
    return df

In [12]:
def laggenerator_min(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_min{i}']=999999
    for x in range(1,i):
        df.loc[:,f'{colname}_min{i}']=np.minimum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_min{i}'])
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_min{i}']=0
    return df

In [13]:
def laggenerator_max(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_max{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_max{i}']=np.maximum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_max{i}'])
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_max{i}']=0
    return df

In [14]:
def lag_generator_all(df, var):
    for i in range(1,6):   #14
        df=laggenerator(i, var,df)

#     for i in range(1,6):  #11
#         df=laggenerator_diff(i, var,df)

#     for i in range(1,6):
#         df=laggenerator_diff6(i, var,df)

    #for i in range(1,11):
     #   df=laggenerator_MA(i, var,df)

#     for i in range(1,14):
#         df=laggenerator_mean(i, 'ZRI_scaled',df)

#     for i in range(1,2):
#         df=laggenerator_ewm(i, var,df)

#     for i in range(1,14):
#         df=laggenerator_min(i, var,df)

#     for i in range(1,14):
#         df=laggenerator_max(i, var,df)
    
    return df

In [15]:
df.columns

Index(['zip', 'City', 'State', 'Metro', 'CountyName', 'zri', 'year', 'month',
       'datetime', 'percent_white', 'percent_black', 'percent_asian',
       'percent_hispanic', 'percent_native_am', 'percent_other_race',
       'percent_0_17', 'percent_18_39', 'percent_40_64', 'percent_65+',
       'percent_rental_units_vacant', 'percent_rental_units_occupied',
       'percent_graduate_deg', 'percent_bachelors', 'percent_associates',
       'percent_highschool', 'percent_less_highschool',
       'percent_commute_public_transport', 'percent_commute_less_30',
       'percent_buildings_less_10_units', 'percent_buildings_10_19_units',
       'percent_buildings_20_49_units', 'percent_buildings_50+_units',
       'percent_commute_30_to_59', 'percent_commute_60_to_89',
       'percent_commute_90_more', 'percent_new_city', 'percent_new_unit',
       'percent_units_owner_occupied', 'median_building_age',
       'income_per_capita', 'poverty_rate', 'total_pop',
       'percent_workforce_unemployed'

In [16]:
#columns to scale:
scale_columns = ['percent_white',
 'percent_black',
 'percent_asian',
 'percent_hispanic',
 'percent_native_am',
 'percent_other_race',
 'percent_0_17',
 'percent_18_39',
 'percent_40_64',
 'percent_65+',
 'percent_rental_units_vacant',
 'percent_rental_units_occupied',
 'percent_graduate_deg',
 'percent_bachelors',
 'percent_associates',
 'percent_highschool',
 'percent_less_highschool',
 'percent_commute_public_transport',
 'percent_commute_less_30',
 'percent_buildings_less_10_units',
 'percent_buildings_10_19_units',
 'percent_buildings_20_49_units',
 'percent_buildings_50+_units',
 'percent_commute_30_to_59',
 'percent_commute_60_to_89',
 'percent_commute_90_more',
 'percent_new_city',
 'percent_new_unit',
 'percent_units_owner_occupied',
 'median_building_age',
 'income_per_capita',
 'poverty_rate',
 'total_pop',
 'percent_workforce_unemployed',
 'percent_work_from_home',
 'median_age',
 'percent_female',
 'gini_index',
 'percent_not_us_citizen',
 'bs_total_stations',
 'bs_total_systems',
 'has_bike_sharing']

In [17]:
for zipcode in df['zip'].unique():
    globals()[f"minmax_features_{zipcode}"]=MinMaxScaler(copy=False)
    df_filtered=df[df['zip']==zipcode]
    df.loc[df['zip']==zipcode,scale_columns]=globals()[f"minmax_features_{zipcode}"].fit_transform(df_filtered[scale_columns])

In [18]:
df.columns

Index(['zip', 'City', 'State', 'Metro', 'CountyName', 'zri', 'year', 'month',
       'datetime', 'percent_white', 'percent_black', 'percent_asian',
       'percent_hispanic', 'percent_native_am', 'percent_other_race',
       'percent_0_17', 'percent_18_39', 'percent_40_64', 'percent_65+',
       'percent_rental_units_vacant', 'percent_rental_units_occupied',
       'percent_graduate_deg', 'percent_bachelors', 'percent_associates',
       'percent_highschool', 'percent_less_highschool',
       'percent_commute_public_transport', 'percent_commute_less_30',
       'percent_buildings_less_10_units', 'percent_buildings_10_19_units',
       'percent_buildings_20_49_units', 'percent_buildings_50+_units',
       'percent_commute_30_to_59', 'percent_commute_60_to_89',
       'percent_commute_90_more', 'percent_new_city', 'percent_new_unit',
       'percent_units_owner_occupied', 'median_building_age',
       'income_per_capita', 'poverty_rate', 'total_pop',
       'percent_workforce_unemployed'

In [19]:
for feat in scale_columns:
    df = laggenerator(12,feat,df)
df.shape


(93672, 94)

In [20]:
for feat in scale_columns:
    df = laggenerator_diff12(1,feat,df)

In [21]:
df.columns

Index(['zip', 'City', 'State', 'Metro', 'CountyName', 'zri', 'year', 'month',
       'datetime', 'percent_white',
       ...
       'total_pop_diff_lag1_lag12',
       'percent_workforce_unemployed_diff_lag1_lag12',
       'percent_work_from_home_diff_lag1_lag12', 'median_age_diff_lag1_lag12',
       'percent_female_diff_lag1_lag12', 'gini_index_diff_lag1_lag12',
       'percent_not_us_citizen_diff_lag1_lag12',
       'bs_total_stations_diff_lag1_lag12', 'bs_total_systems_diff_lag1_lag12',
       'has_bike_sharing_diff_lag1_lag12'],
      dtype='object', length=136)

In [22]:
df

,zip,City,State,Metro,CountyName,zri,year,month,datetime,percent_white,...,total_pop_diff_lag1_lag12,percent_workforce_unemployed_diff_lag1_lag12,percent_work_from_home_diff_lag1_lag12,median_age_diff_lag1_lag12,percent_female_diff_lag1_lag12,gini_index_diff_lag1_lag12,percent_not_us_citizen_diff_lag1_lag12,bs_total_stations_diff_lag1_lag12,bs_total_systems_diff_lag1_lag12,has_bike_sharing_diff_lag1_lag12
0,1013,Chicopee,MA,Springfield,Hampden County,0.000000,2014,1,2014-01-01,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,1013,Chicopee,MA,Springfield,Hampden County,0.012146,2014,2,2014-02-01,0.975159,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,1013,Chicopee,MA,Springfield,Hampden County,0.024291,2014,3,2014-03-01,0.950318,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,1013,Chicopee,MA,Springfield,Hampden County,0.004049,2014,4,2014-04-01,0.925476,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,1013,Chicopee,MA,Springfield,Hampden County,0.004049,2014,5,2014-05-01,0.900635,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93667,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,0.491713,2019,8,2019-08-01,0.082181,...,0.196241,0.076412,0.321393,0.226190,0.122373,0.224307,-0.085030,0.0,0.0,0.0
93668,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,0.458564,2019,9,2019-09-01,0.061635,...,0.189165,0.026560,0.300638,0.202381,0.162180,0.201876,0.033775,0.0,0.0,0.0
93669,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,0.613260,2019,10,2019-10-01,0.041090,...,0.182089,-0.023293,0.279884,0.178571,0.201986,0.179445,0.152581,0.0,0.0,0.0
93670,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,0.406077,2019,11,2019-11-01,0.020545,...,0.175013,-0.073145,0.259129,0.154762,0.241793,0.157015,0.271386,0.0,0.0,0.0


In [25]:
#train=df[(df['year-month']<='2018-12') & (df['year-month']>'2015-01')]

train=df[(df['year-month']<='2018-12') & (df['year-month']>='2014-01')]

train=train.drop('year-month',axis=1)

Y_train=train['zri']
X_train=train.drop('zri',axis=1)

In [26]:
X_train=X_train.drop(['zip','year','month','datetime','City','State','Metro','CountyName'],axis=1)
cols=X_train.columns

In [27]:

# linmod=RandomForestRegressor(n_estimators=500)

# params=({
#      'min_samples_split':[8,12], 
#         'max_features':[0.8,0.9],
#     'min_samples_leaf':[6,8]
#  })

# grid=GridSearchCV(linmod, param_grid=params, cv=fold)
# grid.fit(X_train, Y_train)

# grid.best_params_

In [33]:
fold=TimeSeriesSplit(n_splits=5)


In [29]:
from sklearn.model_selection import RandomizedSearchCV

In [32]:
# rf=RandomForestRegressor()

# params=({
#     'n_estimators':[100,200,300,400,500],
#      'min_samples_split':[2,5,10], 
#         'max_features':[0.3,0.5,0.7,'auto','sqrt'],
#     'min_samples_leaf':[1,10]
#  })

# grid=RandomizedSearchCV(rf, param_distributions=params,n_iter=10,cv=3,n_jobs=-1,verbose=2)
# grid.fit(X_train, Y_train)

# grid.best_params_

# {'n_estimators': 500,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 'sqrt'}


Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'n_estimators': 500,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt'}

In [37]:
grid.best_score_

0.49740102069316244

In [34]:
X_train.shape

(78060, 126)

In [ ]:
#model=RandomForestRegressor(min_samples_split=8, max_features=0.9, n_estimators=500, min_samples_leaf=10)
model = RandomForestRegressor(n_estimators=500,max_features='sqrt')
selector = RFECV(model, step=1, cv=fold, scoring='r2') #'neg_root_mean_squared_error'
selector = selector.fit(X_train, Y_train)


In [ ]:
rf_model.score(X_train, Y_train)

In [ ]:
feature_importances=pd.DataFrame({'Feature':cols,'Importance':rf_model.feature_importances_}).sort_values('Importance', ascending=False)
feature_importances

In [ ]:
#pd.DataFrame({'Feature':X_train.columns,'Coefs':kr_model.coef_}).sort_values('Coefs')

In [ ]:
Y_test=df[(df['year-month']>'2018-12') & (df['year-month']<='2019-12')][['zri']]
predictor_table=df.copy()
X_test=predictor_table[predictor_table['year-month']>'2018-12']

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    
    #run prediction for one month
    X_test=X_test[X_test['year-month']==month]
    
#     enc2_df2 = pd.DataFrame(enc2.transform(X_test[['zip']]).toarray())
#     X_test = X_test.join(enc2_df2, rsuffix='1')
#     X_test=X_test.fillna(0)    
    
    X_test=X_test.drop(['zip','year','month','datetime','City','State','Metro','CountyName','year-month','zri'],axis=1)
    #X_test = pd.DataFrame(normal.transform(X_test))
    
    val=rf_model.predict(X_test)

    #write current month prediction into predictor_table

    predictor_table.loc[predictor_table['year-month']==month,'zri']=val
        
    predictor_table=lag_generator_all(predictor_table, 'zri')
#     for var in predictor_table.columns[5:43]:
#         predictor_table=lag_generator_all(predictor_table, var)
        
    X_test=predictor_table[predictor_table['year-month']>'2018-12']



In [ ]:
#Y_test['ZRI_Yest_scaled']=Y_test['ZRI_scaled']
rstable=X_test[['zip','year','month','datetime','City','State','Metro','CountyName','year-month','zri']].merge(Y_test[['zri']], left_index=True, right_index=True)


In [ ]:
rstable

In [ ]:
for zipcode in rstable['zip'].unique():
    rstable_filtered=rstable[rstable['zip']==zipcode]
    rstable.loc[rstable['zip']==zipcode,'zri_x']=globals()[f"minmax_{zipcode}"].inverse_transform(rstable_filtered[['zri_x']])
    rstable.loc[rstable['zip']==zipcode,'zri_y']=globals()[f"minmax_{zipcode}"].inverse_transform(rstable_filtered[['zri_y']])   
    

In [ ]:
RSME=(sum((rstable['zri_x']-rstable['zri_y'])**2)/len(rstable))**0.5
RSME

In [ ]:
#128 (max without autoregressive)(random forest)

In [ ]:
rstable['residual']=rstable['zri_x']-rstable['zri_y']

In [ ]:
rstable['residual_squared']=rstable['residual'].map(lambda x: x**2)

In [ ]:
rstable2=rstable.groupby('State').agg('mean').sort_values('residual_squared')
rstable2=rstable2.reset_index()

In [ ]:
rstable2

In [ ]:
df['State'].nunique()

In [ ]:
fig = px.choropleth(rstable2,
                    locations='State',
                    color='residual_squared',
                    color_continuous_scale='Viridis',
                    hover_name='State',
                    locationmode='USA-states',
                    scope='usa')

fig.show()

In [ ]:
# filename='RF_6Feature_Auto'
# feature_importances.to_csv(f'models/features_{filename}.csv')
# feature_importances.to_csv(f'models/residuals_{filename}.csv')

In [ ]:
testmonths=list(zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')]['year-month'].drop_duplicates())

R2_test=zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')][['ZRI_scaled','year-month']]
R2_test['ZRI_result']=0

for i,month in enumerate(testmonths):
    R2_test.loc[R2_test['year-month']==month,'ZRI_result']=list(zri3.loc[zri3['year-month']=='2018-12','ZRI_scaled'].apply(lambda x: x*(1+i*0.002)))

RSME_null=(sum((R2_test['ZRI_scaled']-R2_test['ZRI_result'])**2)/len(R2_test))**0.5
RSME_null


In [ ]:
zips=rstable['RegionName'].unique()[0:10]

In [ ]:
px.line(rstable[rstable['RegionName'].isin(zips)], y=['ZRI_scaled_x','ZRI_scaled_y'],x='year-month', color='RegionName')

In [ ]:
zips=zri3['RegionName'].unique()[0:5]
px.line(zri3[zri3['RegionName'].isin(zips)], y=['ZRI_scaled'],x='year-month', color='RegionName')